### Постановка задачи

Просмотр фильмов на оригинальном языке - это популярный и действенный метод обучения иностранным языкам. Важно выбрать фильм, который подходит студенту по уровню сложности, т.е. студент понимал бы 50-70 % диалогов. Чтобы выполнить это условие, преподаватель должен посмотреть фильм и решить, какому уровню он соответствует. Однако это требует больших временных затрат.

Наша задача - разработать ML решение для автоматического определения уровня сложности англоязычных фильмов. 

Программа-максимум:

           - языковая модель, 
           - веб-интерфейс,
           - микросервис.  
           
### Знакомство с исходными данными

В качестве исходных данных нам предоставлены размеченные по уровню сложности фильмы и субтитры к ним, словари английского языка с набором слов по уровню.



In [1]:
import pysrt
import warnings
import pandas as pd
import os
import numpy as np
import nltk
import re
import pickle

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

#nltk.download('stopwords')
#nltk.download('wordnet')
warnings.filterwarnings('ignore')
RANDOM = 123

Сначала взглянем на эти файлы, чтоб понять, как можно было бы с ними вообще работать.

In [2]:
path_A2 = './English_level/English scores/Subtitles_all/A2/The Walking Dead-S01E01-Days Gone Bye.English.srt'

In [3]:
subs = pysrt.open(path_A2)

In [4]:
len(subs)

673

In [5]:
for i in range(0, len(subs), 50):
    print(subs[i])

1
00:00:03,169 --> 00:00:05,171
( bugs chittering )

51
00:06:24,376 --> 00:06:27,045
- And what do you say to that?
- I know what I want to say.

101
00:09:19,207 --> 00:09:22,085 X1:0
Dispatcher:
Unit 1, unit 3, to eastbound Route 18,

151
00:12:53,996 --> 00:12:56,332
Shh shh. That's it. Do you hear me?
Shh shh shh. Okay.

201
00:20:23,438 --> 00:20:25,440
( bugs chittering )

251
00:28:20,153 --> 00:28:23,030
Take a moment,
look how sharp it is.

301
00:31:39,304 --> 00:31:40,388
Yeah.

351
00:34:22,709 --> 00:34:26,086
Man: Don't look.
Get away from the windows.

401
00:38:37,536 --> 00:38:39,705
That's right.

451
00:41:46,000 --> 00:41:47,835
Conserve your ammo.

501
00:47:57,069 --> 00:47:59,071
( growling )

551
00:52:00,594 --> 00:52:02,429
You're all right.
Go on. You're all right.

601
00:57:15,775 --> 00:57:17,485
Good boy.

651
01:05:33,926 --> 01:05:36,595
♪ Of the place I know so well ♪



Чтобы проанализировать слова, нужно из этой структуры убрать тайминг, нумерацию кадров (видимо, это строки, потому что их количество равно длине файла с субтитрами), убрать слова в скобках, потому что в задаче запрос на понимание диалогов, а в скобках не диалоги, убрать знаки препинания и цифры, привести к нижнему регистру.

### Создание единого датасета из предложенных данных

In [6]:

basepath = './English_level/English scores/Subtitles_all/'

df_1 = pd.DataFrame(columns = ['Level', 'Movie', 'Srt'])


for s in ('A2', 'B1', 'B2', 'C1'):
    
    path = os.path.join(basepath, s)
    
    for file in os.listdir(path):
        
        with open(os.path.join(path, file), 'r', encoding='ISO-8859-1') as infile:
            txt = infile.read()
            
        df_1.loc[len(df_1)] = [s, file[:-4], txt]
        


In [7]:
display(df_1)

,Level,Movie,Srt
0,A2,The Walking Dead-S01E01-Days Gone Bye.English,"1\n00:00:03,169 --> 00:00:05,171\n( bugs chitt..."
1,A2,The Walking Dead-S01E02-Guts.English,"1\n00:00:03,045 --> 00:00:05,047\n- ( birds ch..."
2,A2,The Walking Dead-S01E03-Tell It To The Frogs.E...,"1\n00:00:03,003 --> 00:00:04,671\n( thunder ru..."
3,A2,The Walking Dead-S01E04-Vatos.English,"1\n00:00:03,045 --> 00:00:05,422\n( birds chir..."
4,A2,The Walking Dead-S01E05-Wildfire.English,"1\n00:00:03,420 --> 00:00:04,922\n- ( walkie-t..."
...,...,...,...
158,C1,Suits.S03E06.720p.HDTV.x264-mSD,"ï»¿1\n00:00:01,383 --> 00:00:02,751\nI lost Av..."
159,C1,Suits.S03E07.HDTV.x264-mSD,"ï»¿1\n00:00:00,052 --> 00:00:01,352\nPreviousl..."
160,C1,Suits.S03E08.480p.HDTV.x264-mSD,"ï»¿1\n00:00:01,436 --> 00:00:03,028\nI get Ava..."
161,C1,Suits.S03E09.480p.HDTV.x264-mSD,"ï»¿1\n00:00:00,024 --> 00:00:01,478\nPreviousl..."


В первой таблице, собранной из данных папки заказчика с распределением субтитров по уровню языка,  у нас есть полный набор данных - название фильма, его рейтинг, текст субтитров.

In [8]:

df_2 = pd.DataFrame(columns = ['Movie', 'Srt'])
path = os.path.join(basepath, 'Subtitles')
for file in os.listdir(path):
    if file!='.DS_Store':
        
        with open(os.path.join(path, file), 'r', encoding='ISO-8859-1') as infile:
            txt = infile.read()
        df_2.loc[len(df_2)] = [file[:-4], txt]
        


In [9]:
display(df_2)

,Movie,Srt
0,10_Cloverfield_lane(2016),"1\n00:00:55,279 --> 00:01:07,279\n<font color=..."
1,10_things_I_hate_about_you(1999),"1\n00:01:54,281 --> 00:01:55,698\nHey!\n\n2\n0..."
2,Aladdin(1992),"1\n00:00:27,240 --> 00:00:30,879\n<i>Oh, I com..."
3,All_dogs_go_to_heaven(1989),"1\n00:00:12,319 --> 00:00:14,821\nCAPTIONING M..."
4,An_American_tail(1986),"ï»¿1\n00:02:24,080 --> 00:02:26,528\n(INDISTIN..."
...,...,...
110,Warm_bodies(2013),"2\n00:00:26,559 --> 00:00:28,627\n<i>What am I..."
111,Westworld_scenes_of_Dr_Robert_Ford,"1\n00:00:00,000 --> 00:00:21,179\n[Music]\n\n2..."
112,We_are_the_Millers(2013),"1\n00:00:02,400 --> 00:00:03,731\n<i>Oh, my Go..."
113,While_You_Were_Sleeping(1995),"1\n00:02:20,760 --> 00:02:24,720\nLUCY: <i>Oka..."


Во второй таблице, тоже собранной из данных файлов в папке заказчика, но другой, не размеченной по уровню языка, есть названия фильмов и текст субтитров.

In [10]:
df_3 = pd.read_excel('./English_level/English scores/movies_labels.xlsx')

In [11]:
display(df_3)

,id,Movie,Level
0,0,10_Cloverfield_lane(2016),B1
1,1,10_things_I_hate_about_you(1999),B1
2,2,A_knights_tale(2001),B2
3,3,A_star_is_born(2018),B2
4,4,Aladdin(1992),A2/A2+
...,...,...,...
236,236,Matilda(2022),C1
237,237,Bullet train,B1
238,238,Thor: love and thunder,B2
239,239,Lightyear,B2


In [12]:
df_3.drop(columns='id', axis=1)

,Movie,Level
0,10_Cloverfield_lane(2016),B1
1,10_things_I_hate_about_you(1999),B1
2,A_knights_tale(2001),B2
3,A_star_is_born(2018),B2
4,Aladdin(1992),A2/A2+
...,...,...
236,Matilda(2022),C1
237,Bullet train,B1
238,Thor: love and thunder,B2
239,Lightyear,B2


В третьей таблице, предоставленной самим заказчиком, есть названия фильмов и уровень языка.

Теперь нам предстоит объединить эти данные в одну таблицу.

Столбцом для объединения точно будет название фильма. Объединение df_3 с df_2 не составит труда.

In [13]:
df_2_3 = df_3.merge(df_2, on='Movie', how='outer').drop(columns='id')

In [14]:
df_2_3.sample(10)

,Movie,Level,Srt
81,The_Legend_of_Tarzan(2016),C1,"1\n00:02:17,040 --> 00:02:18,724\n<i>Monsieur ..."
43,House_of_Gucci(2021),B2,"1\n00:00:36,769 --> 00:00:41,769\n<font color=..."
70,Ratatouille(2007),B2,"0\n00:00:01,000 --> 00:00:04,000\nDownloaded F..."
48,Knives_out(2019),B2,"1\n00:01:43,157 --> 00:01:45,459\nMorning, Mr...."
228,Suits.S03E04.480pHDTV.x264-mSD,C1,NaN
225,Suits.S03E01.480pHDTV.x264-mSD,C1,NaN
131,Spirit.Stallion.of.the.Cimarron.EN,B1,NaN
36,Good_Will_Hunting(1997),B2,"1\n00:03:34,295 --> 00:03:38,050\nMod f-x-squa..."
223,Suits S04E15 EngSub,C1,NaN
91,The_holiday(2006),B1,"1\n00:01:06,900 --> 00:01:10,700\nI have found..."


И с третьей таблицей.

In [15]:
df_all = df_2_3.merge(df_1, on='Movie', how='outer')

In [16]:
df_all.sample(4)

,Movie,Level_x,Srt_x,Level_y,Srt_y
13,Before_sunrise(1995),"B1, B2","1\n00:03:34,084 --> 00:03:54,084\n<b>Resync By...",NaN,NaN
129,Seven.Worlds.One.Planet.S01E06.2160p.BluRay.Re...,B1,NaN,B1,"ï»¿1\n00:00:44,462 --> 00:00:47,506\nOne conti..."
87,The_cabin_in_the_woods(2012),A2/A2+,"1\n00:01:00,416 --> 00:01:01,983\nIt's hormona...",NaN,NaN
131,Spirit.Stallion.of.the.Cimarron.EN,B1,NaN,B1,"1\n00:00:36,883 --> 00:00:46,554\n''Spirit Sta..."


Есть пробелы в данных.

In [17]:
df_all.isna().sum()

Movie        0
Level_x     49
Srt_x      171
Level_y    127
Srt_y      127
dtype: int64

In [18]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290 entries, 0 to 289
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Movie    290 non-null    object
 1   Level_x  241 non-null    object
 2   Srt_x    119 non-null    object
 3   Level_y  163 non-null    object
 4   Srt_y    163 non-null    object
dtypes: object(5)
memory usage: 11.5+ KB


Столбцы уровней и субтитров будем объединять.

In [19]:
df_all['Level_x'].unique()

array(['B1', 'B2', 'A2/A2+', 'C1', 'B1, B2', 'A2/A2+, B1', 'A2', nan],
      dtype=object)

In [20]:
df_all['Level_y'].unique()

array([nan, 'A2', 'B1', 'B2', 'C1'], dtype=object)

Иногда указывают несколько уровней языка для одного фильма (А2, В1). Пока я оставлю самый высокий, это проще. Можно отложить эти сомнительные фильмы и потом определить обученной моделью их уровень. Уровень А2/А2+ заменяем на А2, потому что у нас не стоит задачи в градации подуровней внутри уровня.

In [21]:
df_all['Level_x'] = df_all['Level_x'].replace('A2/A2+', 'A2')
df_all['Level_x'] = df_all['Level_x'].replace('A2/A2+, B1', 'B1')

In [22]:
df_all['Level_x'] = df_all['Level_x'].replace('B1, B2', 'B2')

In [23]:
df_all['Level_x'].unique()

array(['B1', 'B2', 'A2', 'C1', nan], dtype=object)

In [24]:
print('Строк с разными значениями в столбцах Label_x и Label_y:',\
    len(df_all.loc[(df_all['Level_x'].notna())&(df_all['Level_y'].notna()) & (df_all['Level_x']!=df_all['Level_y'])]))

Строк с разными значениями в столбцах Label_x и Label_y: 0


Значит, просто заполним пропуски переносом значений из одного стобца в другой.

In [25]:
df_all['Level']=df_all['Level_x'].fillna(df_all['Level_y'])

In [26]:
df_all['Level'].unique()

array(['B1', 'B2', 'A2', 'C1', nan], dtype=object)

In [27]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290 entries, 0 to 289
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Movie    290 non-null    object
 1   Level_x  241 non-null    object
 2   Srt_x    119 non-null    object
 3   Level_y  163 non-null    object
 4   Srt_y    163 non-null    object
 5   Level    281 non-null    object
dtypes: object(6)
memory usage: 13.7+ KB


Также проверим, есть ли несовпадения по субтитрам.

In [28]:
print('Строк с разными значениями в столбцах Srt_x и Srt_y:',\
    len(df_all.loc[(df_all['Srt_x'].notna())&(df_all['Srt_y'].notna()) & (df_all['Srt_x']!=df_all['Srt_y'])]))

Строк с разными значениями в столбцах Srt_x и Srt_y: 0


In [29]:
df_all['Srt']=df_all['Srt_x'].fillna(df_all['Srt_y'])

In [30]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290 entries, 0 to 289
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Movie    290 non-null    object
 1   Level_x  241 non-null    object
 2   Srt_x    119 non-null    object
 3   Level_y  163 non-null    object
 4   Srt_y    163 non-null    object
 5   Level    281 non-null    object
 6   Srt      282 non-null    object
dtypes: object(7)
memory usage: 16.0+ KB


In [31]:
df_all.columns

Index(['Movie', 'Level_x', 'Srt_x', 'Level_y', 'Srt_y', 'Level', 'Srt'], dtype='object')

In [32]:
df = df_all.copy().drop(columns=['Level_x', 'Srt_x', 'Level_y', 'Srt_y'])

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290 entries, 0 to 289
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Movie   290 non-null    object
 1   Level   281 non-null    object
 2   Srt     282 non-null    object
dtypes: object(3)
memory usage: 6.9+ KB


Если пропуски в разных строках, то удалится в общей сложности 17 строк. Пока восстанавливать пропущенные значения не буду - надо смочь выполнить работу в минимальном объёме.

In [34]:
df_grade = df.loc[(df['Level'].isna())&(df['Srt'].notna())]

Отделяю фильмы с субтитрами, но неопределённым уровнем языка.

In [35]:
df_grade

,Movie,Level,Srt
241,Breaking_Bad_The_Movie(2017),NaN,"1\n00:00:05,299 --> 00:00:08,220\nwhat hi good..."
242,Bren╨Т.Brown.The.Call.to.Courage.2019.720.NF.7...,NaN,"ï»¿1\n00:00:07,216 --> 00:00:09,676\n[presente..."
243,Casper,NaN,"1\n00:00:18,040 --> 00:00:21,870\n[Music]\n\n2..."
244,Gogo_Loves_English,NaN,"1\n00:00:01,130 --> 00:00:18,359\n[Music]\n\n2..."
245,Harry_Potter_and_the_philosophers_stone(2001),NaN,"1\n00:01:22,065 --> 00:01:27,070\nI should've ..."
246,Pride_and_Prejudice,NaN,"1\n00:00:20,020 --> 00:00:42,009\n[Music]\n\n2..."
247,The_Ghost_Writer,NaN,"1\n00:00:04,670 --> 00:00:18,579\n[Music]\n\n2..."
248,Up(2009),NaN,"1\n00:00:46,040 --> 00:00:50,960\n{\i1}Movieto..."
249,Westworld_scenes_of_Dr_Robert_Ford,NaN,"1\n00:00:00,000 --> 00:00:21,179\n[Music]\n\n2..."


In [36]:
df = df.dropna(subset=['Level', 'Srt'])

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 273 entries, 0 to 289
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Movie   273 non-null    object
 1   Level   273 non-null    object
 2   Srt     273 non-null    object
dtypes: object(3)
memory usage: 8.5+ KB


Теперь посмотрим на дубли. Найдём - удалим.

In [38]:
df.duplicated().sum()

3

In [39]:
df.drop_duplicates()
df = df.reset_index()
df.head(8)

,index,Movie,Level,Srt
0,0,10_Cloverfield_lane(2016),B1,"1\n00:00:55,279 --> 00:01:07,279\n<font color=..."
1,1,10_things_I_hate_about_you(1999),B1,"1\n00:01:54,281 --> 00:01:55,698\nHey!\n\n2\n0..."
2,2,A_knights_tale(2001),B2,"1\n00:00:15,089 --> 00:00:21,229\nResync: Xenz..."
3,3,A_star_is_born(2018),B2,"1\n00:00:17,610 --> 00:00:22,610\n- <i><font c..."
4,4,Aladdin(1992),A2,"1\n00:00:27,240 --> 00:00:30,879\n<i>Oh, I com..."
5,5,All_dogs_go_to_heaven(1989),A2,"1\n00:00:12,319 --> 00:00:14,821\nCAPTIONING M..."
6,6,An_American_tail(1986),A2,"ï»¿1\n00:02:24,080 --> 00:02:26,528\n(INDISTIN..."
7,7,Babe(1995),A2,"0\n00:02:31,114 --> 00:02:34,316\nThis is a ta..."


In [40]:
df.drop('index', axis=1)

,Movie,Level,Srt
0,10_Cloverfield_lane(2016),B1,"1\n00:00:55,279 --> 00:01:07,279\n<font color=..."
1,10_things_I_hate_about_you(1999),B1,"1\n00:01:54,281 --> 00:01:55,698\nHey!\n\n2\n0..."
2,A_knights_tale(2001),B2,"1\n00:00:15,089 --> 00:00:21,229\nResync: Xenz..."
3,A_star_is_born(2018),B2,"1\n00:00:17,610 --> 00:00:22,610\n- <i><font c..."
4,Aladdin(1992),A2,"1\n00:00:27,240 --> 00:00:30,879\n<i>Oh, I com..."
...,...,...,...
268,Virgin.River.S01E06.INTERNAL.720p.WEB.x264-STRiFE,B2,"1\n00:00:18,852 --> 00:00:19,852\nHey.\n\n2\n0..."
269,Virgin.River.S01E07.INTERNAL.720p.WEB.x264-STRiFE,B2,"1\n00:00:10,468 --> 00:00:13,178\nAre you sure..."
270,Virgin.River.S01E08.INTERNAL.720p.WEB.x264-STRiFE,B2,"1\n00:00:07,382 --> 00:00:10,012\nTwo IVs in p..."
271,Virgin.River.S01E09.INTERNAL.720p.WEB.x264-STRiFE,B2,"1\n00:00:16,474 --> 00:00:18,024\nOh.\n\n2\n00..."


In [41]:
del df_1, df_2, df_3, df_2_3, df_all

Удалила уже ненужные переменные.

### Обработка субтитров

Необходимо преобразовать текст в числовую форму, удобную для моделирования.

Избавимся от лишних символов, стоп-слов и скобок.

In [42]:
stop = stopwords.words('english') + ["im", "ill", "oh", "hey"]

Нужно оставить только текст, поэтому с помощью регулярных выражений удалим ненужное - небуквенные символы, содержимое скобок, html-теги, переносы, пробелы больше одного подряд.

In [43]:
del_n = re.compile('\n')                  # перенос каретки
del_tags = re.compile('<[^>]*>')          # html-теги
del_brackets = re.compile('\([^)]*\)')    # содержимое круглых скобок
clean_text = re.compile('[^а-яa-z\s]')  # все небуквенные символы кроме пробелов
del_spaces = re.compile('\s{2,}')         

def prepare_text(text):
    text = del_n.sub(' ', str(text).lower())
    text = del_tags.sub('', text)
    text = del_brackets.sub('', text)
    res_text = clean_text.sub('', text)
    res_text = res_text.lower()  #перевод в нижний регистр
    return del_spaces.sub(' ',res_text)


Следующая функция разбивает текст субтитров на слова и удаляет те, которые встречаются в списке стоп-слов.
Пока не очень понимаю, пригодится ли длина первоначального текста, но я её посчитаю. И длину образовавшегося массива слов.

In [44]:
def df_stop_words(Sr, stop):

    for itm in range(len(Sr)):
        line = Sr[itm]
        line = prepare_text(line)
        line = [w for w in line.split() if w not in stop]
        Sr[itm] = line
    return Sr

In [45]:
df['Srt'] = df_stop_words(df['Srt'], stop)

In [46]:
df.head(8)

,index,Movie,Level,Srt
0,0,10_Cloverfield_lane(2016),B1,"[fixed, synced, bozxphd, enjoy, flick, ben, ph..."
1,1,10_things_I_hate_about_you(1999),B1,"[right, cameron, go, nine, schools, years, arm..."
2,2,A_knights_tale(2001),B2,"[resync, xenzainef, retail, help, hes, due, li..."
3,3,A_star_is_born(2018),B2,"[synced, corrected, mrcjnthn, get, black, eyes..."
4,4,Aladdin(1992),A2,"[come, land, faraway, place, caravan, camels, ..."
5,5,All_dogs_go_to_heaven(1989),A2,"[captioning, made, possible, mgm, home, entert..."
6,6,An_American_tail(1986),A2,"[mama, tanya, fievel, stop, twirling, twirling..."
7,7,Babe(1995),A2,"[tale, aboutan, unprejudiced, heart, changed, ..."


Теперь проведём лемматизацию - приведём слова к корневой форме.

In [47]:
#stemmer = WordNetLemmatizer()
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

In [48]:
def lemmatise_srt(Sr):
    for itm in range(len(Sr)):
        line = Sr[itm]
        line = [porter.stem(word) for word in line]
        Sr[itm] =  str(line)
    return Sr    

In [49]:
df['Srt'] = lemmatise_srt(df['Srt'])

In [50]:
df.drop(columns=['index'])  #.to_csv('movie_data.csv', index=False)

,Movie,Level,Srt
0,10_Cloverfield_lane(2016),B1,"['fix', 'sync', 'bozxphd', 'enjoy', 'flick', '..."
1,10_things_I_hate_about_you(1999),B1,"['right', 'cameron', 'go', 'nine', 'school', '..."
2,A_knights_tale(2001),B2,"['resync', 'xenzainef', 'retail', 'help', 'he'..."
3,A_star_is_born(2018),B2,"['sync', 'correct', 'mrcjnthn', 'get', 'black'..."
4,Aladdin(1992),A2,"['come', 'land', 'faraway', 'place', 'caravan'..."
...,...,...,...
268,Virgin.River.S01E06.INTERNAL.720p.WEB.x264-STRiFE,B2,"['wouldnt', 'chainsaw', 'easier', 'yeah', 'muc..."
269,Virgin.River.S01E07.INTERNAL.720p.WEB.x264-STRiFE,B2,"['sure', 'cant', 'convinc', 'stay', 'right', '..."
270,Virgin.River.S01E08.INTERNAL.720p.WEB.x264-STRiFE,B2,"['two', 'iv', 'place', 'antecubit', 'fossa', '..."
271,Virgin.River.S01E09.INTERNAL.720p.WEB.x264-STRiFE,B2,"['yeah', 'okay', 'right', 'yeah', 'okay', 'hi'..."


###### Теперь приступим к векторизации.

Поскольку в будущем модель должна будет работать с неопределённым набором слов из текста, для векторизации необходимо создать некоторый словарь, который определит количество и состав фичей в модели. Пока я строю одну модель на все уровни, соберу один словарь. В дальнейшем можно обучить несколько моделей, на каждый уровень языка свою, тогда словари будут не такие большие и только на свой уровень. Но это в будущем.

Нам предоставлены словари с градацией по уровням, американские и британские в формате pdf. У них есть текстовый слой, скопирую вручную в отдельный файлик 'dict.xlsx', потом его преобразую.

In [51]:
dict_eng = pd.read_excel('dict.xlsx')
dict_eng.columns = ['words']

In [52]:
dict_eng

,words
0,ability n.
1,able adj.
2,accept v.
3,accident n.
4,according to prep.
...,...
4434,"worship n., v."
4435,worthwhile adj.
4436,worthy adj.
4437,yell v.


Нужно почистить слова от частей речи, предлогов и также лемматизировать. Дубли после этого удалить.

In [53]:
stop = stop + ['ah', 'adj', 'n', 'v', 'prep', 'conj', 'pron', 'modal v', 'adv', 'adjadv', 'detpron', 'modal', 'exclam', 'number', 'nadj' ]


In [54]:
dict_eng['words'] = df_stop_words(dict_eng['words'], stop)

In [55]:
dict_eng['words'] = lemmatise_srt(dict_eng['words'])

In [56]:
dict_eng.duplicated().sum()

977

In [57]:
dict_eng = dict_eng.drop_duplicates().reset_index().drop(columns=['index'])

In [58]:
def to_words(Sr):
    for itm in range(len(Sr)):
        line = Sr[itm]
        if line=='':
            Sr[itm]=np.nan
        else:
            Sr[itm] = str(prepare_text(line))
            
    return Sr

In [59]:
dict_eng['words'] = to_words(dict_eng['words'])

In [60]:
dict_eng.isna().sum()

words    0
dtype: int64

In [61]:
dict_eng['words'] = to_words(dict_eng['words'])

In [62]:
dict_eng.isna().sum()

words    1
dtype: int64

По непонятной мне причине не опустошается строка с первого раза. Что-то я делаю не так.

In [63]:
dict_eng = dict_eng.dropna()

In [64]:
#dict_eng = dict_eng.loc[dict_eng.index!=14]

Теперь нужно сделать из этого словарь.

In [65]:
eng_voc = np.array(dict_eng['words'])
ind_voc = [i for i in range(len(dict_eng))]
eng_vocabulary = dict(zip(eng_voc, ind_voc))
eng_vocabulary

{'abil': 0,
 'abl': 1,
 'accept': 2,
 'accid': 3,
 'accord': 4,
 'achiev': 5,
 'act': 6,
 'activ': 7,
 'actual': 8,
 'adult': 9,
 'advantag': 10,
 'adventur': 11,
 'advertis': 12,
 'affect': 13,
 'airlin': 14,
 'aliv': 15,
 'right': 16,
 'allow': 17,
 'almost': 18,
 'alon': 19,
 'along': 20,
 'alreadi': 21,
 'altern': 22,
 'although': 23,
 'among': 24,
 'amount': 25,
 'analyz': 26,
 'ancient': 27,
 'ankl': 28,
 'anybodi': 29,
 'anymor': 30,
 'anyway': 31,
 'anywher': 32,
 'app': 33,
 'appear': 34,
 'appli': 35,
 'architect': 36,
 'architectur': 37,
 'argu': 38,
 'argument': 39,
 'armi': 40,
 'arrang': 41,
 'asleep': 42,
 'assist': 43,
 'athlet': 44,
 'attack': 45,
 'attend': 46,
 'attent': 47,
 'attract': 48,
 'audienc': 49,
 'author': 50,
 'avail': 51,
 'averag': 52,
 'avoid': 53,
 'award': 54,
 'aw': 55,
 'back': 56,
 'background': 57,
 'badli': 58,
 'base': 59,
 'bean': 60,
 'bear': 61,
 'beat': 62,
 'beef': 63,
 'behav': 64,
 'behavior': 65,
 'belong': 66,
 'belt': 67,
 'benefit': 

In [66]:
pkl_filename = "pickle_dict.pkl" 
with open(pkl_filename, 'wb') as file: 
    pickle.dump(eng_vocabulary, file)

Этот словарь передам модели в приложение.

Может быть так, что в предыдущих обработках какие-то слова испортились или в тексте встречаются опечатки, поэтому в качестве параметра векторизации возьму минимальное число документов, в которых встречается слово, - два.

In [67]:
count = CountVectorizer(min_df=2, vocabulary=eng_vocabulary) 

bag = count.fit_transform(df['Srt'])

In [68]:
tfidfconverter = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
bag = tfidfconverter.fit_transform(bag).toarray()

Преобразование из слов в цифры закончилось. Теперь эту матрицу надо соотнести с таргетом и построить на их основе модель предсказания.

### Построение модели

#### Обучающая и тестовая выборки

Я собираюсь использовать кросс-валидацию, поэтому валидационная выборка  не нужна, только тестовая и обучающая.

Разделю в соотношении 70/30.

In [69]:
features = bag
target = df['Level']

In [70]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.3, random_state=RANDOM)

In [71]:
param_grid = {'penalty': ['l1','l2'],               
              'solver': ['lbfgs', 'newton-cg'],              
              'C': [1.0, 10.0, 100.0]}
              

model = LogisticRegression(random_state=RANDOM, n_jobs=1, multi_class='auto')

gs= GridSearchCV(model, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=-1)

In [72]:
gs.fit(features_train, target_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(cv=5, estimator=LogisticRegression(n_jobs=1, random_state=123),
             n_jobs=-1,
             param_grid={'C': [1.0, 10.0, 100.0], 'penalty': ['l1', 'l2'],
                         'solver': ['lbfgs', 'newton-cg']},
             scoring='accuracy', verbose=1)

In [73]:
print('Best parameter set: %s ' % gs.best_params_)
print('CV Accuracy: %.3f' % gs.best_score_)

Best parameter set: {'C': 100.0, 'penalty': 'l2', 'solver': 'lbfgs'} 
CV Accuracy: 0.649


In [74]:
best_model = gs.best_estimator_
print('Test Accuracy: %.3f' % best_model.score(features_test, target_test))

Test Accuracy: 0.695


In [75]:
predicted = best_model.predict(features_test)
df_test = pd.DataFrame()
df_test['Level_target'] = target_test
df_test = df_test.reset_index()
df_test['Level_pred'] = pd.Series(predicted)
df_test.sample(8)

,index,Level_target,Level_pred
20,204,C1,C1
37,258,B2,B1
52,36,B2,B2
0,167,B2,B2
67,194,B2,B2
64,242,B2,B2
44,95,B2,B1
10,52,B1,B1


In [76]:
df_test['Level_pred'].unique()

array(['B2', 'B1', 'C1', 'A2'], dtype=object)

Можем увидеть, что модель худо-бедно предсказывает уровень сложности текста.

Теперь выложу её в отдельный файл, чтоб использовать в приложении.

In [77]:
pkl_filename = "pickle_model.pkl" 
with open(pkl_filename, 'wb') as file: 
    pickle.dump(best_model, file)

###  Итог

В тетради я объединила три таблицы субтитров, отредактировала текст, убрав ненужные символы, цифры, пробелы, оставила только грамматические основы. Создала словарь для векторизации любого входящего текста, не только обучающего модель. Построила на частотности появления основ в документе и во всём объёме текстов векторы для модели, опираясь на словарь. Обучила модель линейной регрессии.

In [79]:
print('Best parameter set: %s ' % gs.best_params_)
print('Test Accuracy: %.3f' % best_model.score(features_test, target_test))

Best parameter set: {'C': 100.0, 'penalty': 'l2', 'solver': 'lbfgs'} 
Test Accuracy: 0.695


Словарь и модель записала в отдельные файлы для предоставления их в приложение. 

